In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import csv

In [ ]:
headers = {
    'accept' :'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'user_agent' : 'Mozlla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
CSV = 'games.csv'
CSV_2 = 'prices.csv'

In [ ]:
def get_content(params=''):
  host = 'https://steampricehistory.com'
  url = 'https://steampricehistory.com/popular?page='
  r = requests.get(url, headers=headers, params=params)
  soup = bs(r.text, 'html.parser')
  items = soup.find_all('tr')[1:]
  games = []
  total_links = []
  
  for item in items:
    total_links.append(item.find('a').get('href'))
    games.append(
        {
            'title' : item.find('a', class_='app-link').get_text(strip=True),
            'recommended' : item.find_all('td')[4].get_text(strip=True),
            'current_price' : item.find_all('td')[2].get_text(strip=True),
            'discount' : item.find_all('td')[3].get_text(strip=True)
            }
            )
  return games, total_links

Здесь с главной страницы собираются название игры, количество рекоммендаций, текущая цена и скидка. На выходе получаем список с играми и ссылки на каждую игру, чтоб потом их запарсить.

In [ ]:
def get_prices(total_links):
  game_data = []
  for link in total_links:
    url_game = link
    req = requests.get(url_game, headers=headers)
    borsch = bs(req.text, 'html.parser')
    prices = borsch.find('table', class_ = 'breakdown-table').find_all('tr')[1:]
    single_game = []
    for price in prices:
      single_game.append(price.find_all('td')[1].get_text(strip=True))
    game_data.append(single_game)
  return game_data

Парсинг страницы каждой игры, получаем все изменения цен на каждую игру как список, внутри которого другие списки

In [ ]:
def csv_saver_common(items, path):
  with open(path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter = ';')
    titles = ['Game title', 'Amount of recommendations', 'Current price', 'Current discount']
    writer.writerow(titles)
    for item in items:
      writer.writerow([item['title'], item['recommended'], item['current_price'], item['discount']])

In [ ]:
def csv_saver_prices(prices, path):
  with open(path, 'w', newline='') as file:
    writer = csv.writer(file, delimiter = ';')
    title = ['Price changes']
    writer.writerow(list(range(1, 51, 1)))
    for price in prices:
      writer.writerow(price)

Данные записываются в csv

In [ ]:
def parser():
  pages = list(range(1, 213, 1))
  games = []
  game_prices = []
  for page in pages:
    games.extend(get_content(params ={'page' : page})[0])
    link = get_content(params ={'page' : page})[1]
    game_prices.append(get_prices(link))
  csv_saver_common(games, CSV)
  csv_saver_prices(game_prices, CSV_2)

In [ ]:
parser()

Запускаются все функции и листаются страницы сайта

In [ ]:
games_df = pd.read_csv('games.csv', sep=';')
prices_df = pd.read_csv('prices.csv', sep=';')
prices_df = prices_df.T
games_df

,Game title,Amount of recommendations,Current price,Current discount
0,Counter-Strike: Global Offensive,"2,999,135",$0.00,0
1,PUBG: BATTLEGROUNDS,"1,382,744",$29.99,0
2,Rainbow Six Siege - Pro League Mute Set,"710,559",$19.99,0
3,Rainbow Six Siege - Pro League Bandit Set,"710,559",$19.99,0
4,Rainbow Six Siege - Pro League Thermite Set,"710,559",$19.99,0
...,...,...,...,...
10595,Micron,101,$2.99,0
10596,Haunted Past: Realm of Ghosts,101,$4.99,0
10597,Last Knight: Rogue Rider Edition,101,$7.99,0
10598,Saints Row IV - Hey Ash Whatcha Playin? Pack,101,$1.99,0


In [ ]:
prices = prices_df[[0]] 
for i in range(1, len(prices_df.columns)):
  prices = pd.concat([prices, prices_df.iloc[:, i].T.to_frame().rename(columns = {i : 0})])
prices = prices.reset_index().drop('index', axis=1)

In [ ]:
total = games_df.join(prices)
total = total.rename(columns = {0 : 'Price changes'})

In [ ]:
total.to_csv('total.csv', sep=';', index=False)

Получение итоговой таблицы